In [13]:
import time
from llama_cpp import Llama

In [14]:
MODEL_PATH = "./models/kanana-nano-2.1b-instruct.Q4_K_M.gguf" 

TERMS_DOC = """
제1조 (목적)
본 약관은 고객의 카드 분실 신고 시 즉시 카드 사용을 정지하여 부정 사용을 방지하고, 고객의 재산상 손실을 최소화하기 위함입니다.

제2조 (적용 범위)
본 약관은 테디카드가 발행한 모든 신용카드 및 체크카드에 적용됩니다.

제3조 (신고 접수 및 처리)
1. 고객이 카드 분실 신고를 하는 즉시, 카드사는 해당 카드의 사용을 정지합니다.
2. 사용 정지는 신고 접수 시점으로부터 즉시 시스템에 반영되며, 처리 시간은 1-2분 이내입니다.
3. 분실 신고 접수 후에는 해당 카드로 거래가 불가능합니다.

제4조 (본인 확인)
1. 분실 신고 시 본인 확인을 위해 다음 정보를 요청할 수 있습니다:
   - 생년월일
   - 카드번호 뒷자리
   - 최근 거래 내역
2. 본인 확인 실패 시 추가 인증 절차가 필요합니다.

제5조 (부정 사용 확인)
1. 분실 신고 접수 시, 최근 48시간 내 거래 내역을 확인합니다.
2. 본인이 사용하지 않은 거래가 발견될 경우, 즉시 조사 절차를 진행합니다.

제6조 (고객의 책임)
1. 고객은 카드 분실 사실을 인지한 즉시 카드사에 신고해야 합니다.
2. 신고 지연으로 인한 부정 사용은 고객의 책임이 될 수 있습니다.

제7조 (법적 근거)
본 약관은 여신전문금융업법 제16조(선불카드 등의 발행 및 관리)에 따릅니다.
"""

In [15]:
PAST_CASE = """
상담사: 상담원 홍길동입니다.
손님: 네, 저 테디카드 문의좀 드릴려고요.
상담사: 네.
상담사: 네, 고객님. 그럼 본인 확인 후 안내를 해드리겠습니다. 고객님 성함과 생년월일 말씀해 주시겠어요?
손님: 김철수이고요, 0912요.
상담사: 네.
상담사: 핸드폰 번호는 어떻게 되세요?
손님: 010-1234-5678요.
상담사: 테디카드 결제 은행은 어느 은행이세요?
손님: 베어은행이요.
상담사: 아 네, 감사합니다. 말씀해 주세요.
손님: 제가 이게 이제
상담사: 네.
손님: 어 기간이 다 돼서 제가 교체 발급인줄 알고 받아서 그런데 지금 보니까 카드가 2개가 다 사용이 가능한 거였네요.
상담사: 네.
상담사: 카드요?
손님: 네, 테디카드.
상담사: 지금 보니까 고객님 테디체크카드 지금 이거 한 장 있는 걸로 보이는데 고객님.
손님: 예.
상담사: 갱신돼서 나가는 카드
손님: 어 그
손님: 아 그러면 어제까지 제가 갱신되기 전에 그 어 좀전인가 어제까지 제가
상담사: 네.
손님: 4856으로 끝나는 카드를 사용했어요.
상담사: 네, 고객님.
손님: 그 요거 해지가 됐다는 말씀이시죠?
상담사: 그리고 이제 그 카드 유효 기간이 다 되잖아요?
손님: 12월이거든요, 예.
상담사: 아 네네 다 돼 가잖아요? 그래서 갱신돼서 새카드 나갔으니까 이제 기존 거는 잘라서 버려 주시고요, 새 카드 써주시면 되세요.
손님: 예.
손님: 네, 그럼 기존 건 해지가 된 거죠?
상담사: 네, 잘라서 버려 주시면 되세요. 이제 새 카드 쓰시면 그걸로 계속 이용해주시면 되시는 부분입니다.
손님: 네.
손님: 예, 알겠습니다.
상담사: 네, 감사합니다.
손님: 예.
상담사: 네.
"""

In [ ]:
SYSTEM_PROMPT = """
당신은 신입 상담사를 코칭하는 'AI 상담 가이드'입니다.
제공된 <상품 약관>과 <과거 상담 사례>를 참고하여, 현재 접수된 고객 문의에 대한 최적의 대응 멘트를 제안하십시오.

[지시사항]
1. <상품 약관>의 사실관계(Fact)를 철저히 준수하십시오. 없는 내용을 지어내면 안 됩니다.
2. <과거 상담 사례>의 말투와 공감하는 태도(Tone & Manner)를 모방하십시오.
3. 출력 형식은 다음과 같아야 합니다:
   - [추천 응대 멘트]: 고객에게 직접 말할 스크립트
   - [확인 필요한 질문]: 고객 정보를 더 확인해야 할 경우 물어볼 질문 (필요 없다면 생략)
   - [근거 조항]: 답변의 근거가 되는 약관 조항

<상품 약관>
{terms}

<과거 상담 사례>
{past_case}
"""

class ConsultationAdvisor:
    def __init__(self, model_path):
        print(f"🔄 Loading Advisor Model...")
        try:
            self.llm = Llama(
                model_path=model_path,
                n_gpu_layers=-1,
                n_ctx=4096,
                verbose=False
            )
            print("✅ Model Loaded.")
        except Exception as e:
            print(f"❌ Load Failed: {e}")
            self.llm = None

    def suggest_response(self, current_query):
        if not self.llm: return

        # 프롬프트 조립
        formatted_system = SYSTEM_PROMPT.format(terms=TERMS_DOC, past_case=PAST_CASE)
        
        messages = [
            {"role": "system", "content": formatted_system},
            {"role": "user", "content": f"현재 고객 문의: {current_query}"}
        ]

        output = self.llm.create_chat_completion(
            messages=messages,
            max_tokens=512,
            temperature=0.2, # 약관을 다루므로 창의성을 낮춤
        )
        
        response = output['choices'][0]['message']['content']
        print("\r" + " " * 20 + "\r", end="") # Thinking 지우기
        return response

In [17]:
def main():
    advisor = ConsultationAdvisor(MODEL_PATH)
    if not advisor.llm: return

    # 테스트할 현재 문의 상황들
    current_queries = [
        "이번에 받은 호텔 뷔페 바우처, 지방에 사는 저희 부모님께 드리고 싶은데 가능한가요?", # 양도 불가 규정 체크
        "제가 카드를 해지하려고 하는데, 아직 안 쓴 바우처가 있거든요. 이거 현금으로 돌려받을 수 있나요?", # 해지/환불 규정 체크
        "항공권 바우처 쓰려는데, 와이프랑 같이 가면 둘 다 할인되나요?" # 동반자 규정 체크
    ]

    for query in current_queries:
        suggestion = advisor.suggest_response(query)
        print(f"Q. {query}")
        print(suggestion)

if __name__ == "__main__":
    main()

🔄 Loading Advisor Model...


llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


✅ Model Loaded.
Q. 이번에 받은 호텔 뷔페 바우처, 지방에 사는 저희 부모님께 드리고 싶은데 가능한가요?
### [추천 응대 멘트]
"안녕하세요! 고객님, 테디카드 고객님들께는 특별한 혜택을 제공해 드리기 위해 최선을 다하고 있습니다. 이번에 받으신 호텔 뷔페 바우처를 부모님께 전달하고 싶으신 마음은 정말 따뜻하고 좋은 생각입니다. 현재 저희 시스템에서는 바우처를 다른 사람에게 양도하거나 전달하는 기능을 지원하지 않습니다. 하지만 부모님을 위한 특별한 선물로 생각하시고, 부모님께서 직접 사용하실 수 있도록 도와드릴 수 있는 방법을 찾아보겠습니다. 우선, 부모님께서 직접 사용하실 수 있도록 안내해 드리겠습니다. 추가로 도움이 필요하시면 언제든지 말씀해 주세요. 감사합니다."

### [확인 필요한 질문]
- 부모님의 연락처와 이름을 확인하여 바우처를 전달할 방법을 구체적으로 논의합니다.
- 부모님께서 직접 사용 가능한지 여부를 확인합니다.

### [근거 조항]
- **제3조 (신고 접수 및 처리)**: 분실 신고 시 카드의 사용을 정지할 수 있으며, 부정 사용을 방지하기 위해 시스템에 즉시 반영합니다.
- **제4조 (본인 확인)**: 분실 신고 시 본인 확인을 위해 필요한 정보를 요청할 수 있으며, 본인 확인 실패 시 추가 인증 절차가 필요합니다.
- **제6조 (고객의 책임)**: 고객은 카드 분실 사실을 인지한 즉시 카드사에 신고해야 하며, 신고 지연으로 인한 부정 사용은 고객의 책임이 될 수 있습니다.

이와 같은 조항을 근거로 고객에게 바우처 전달 방법을 안내하고, 추가적인 확인 절차를 통해 고객의 신뢰를 유지할 수 있습니다.
Q. 제가 카드를 해지하려고 하는데, 아직 안 쓴 바우처가 있거든요. 이거 현금으로 돌려받을 수 있나요?
### [추천 응대 멘트]
"안녕하세요, 고객님. 테디카드 고객님께서 바우처를 현금으로 돌려받을 수 있는지 문의해 주셨군요. 본 약관에 따르면, 카드 해지 시 바우처는 일반적으로 카드사에서 보유하고 있는 자산으